In [55]:
import pandas as pd
import numpy as np
import os
import os.path as path
import re

In [56]:
data_path = "./data/정보대 SW contest/279.국내 여행로그 데이터(서부권)/01.데이터/Training/02.라벨링데이터/TL_csv"

In [57]:
# df = pd.read_csv("./data/정보대 SW contest/279.국내 여행로그 데이터(서부권)/01.데이터/Training/02.라벨링데이터/TL_csv/tn_visit_area_info_방문지정보_C.csv")
# print(df.columns)
# df['Y_COORD'] = df['Y_COORD'].astype("str")
# df['Y_COORD'] = df['Y_COORD'].apply(lambda x: ''.join(re.findall(r'\d+', x)))
# df.loc[df['Y_COORD']=="",'Y_COORD'] = None
# df['Y_COORD'].astype("float")


In [58]:
# csv_name_list = os.listdir(data_path)

In [59]:
# print(csv_name_list)

In [60]:
# for csv_name in csv_name_list:
#     print("===================={}===================".format(csv_name))
#     df = pd.read_csv(path.join(data_path, csv_name))
#     print(df.describe())

In [61]:
def print_isna(df):
    print(pd.isna(df).sum())

# 학습 데이터 셋 만들기

# INPUT

> 소득
    - INC || INCOME, HOUSE_INCOME <========== tn_traveller_master_여행객 Master_C 

    특이사항 - 소득 값이 두가지 존재

    INCOMDE사용으로 일단 대체

		-> 기본값 0
    
> 미션
    - MIS || TRAVEL_MISSION(==TRAVEL_PURPOSE) <========== tn_travel_여행_C

    특이사항 - 여행 미션내 값들이 여러개 존재

    3개로 중복 없이

		-> 기본값 0

> 여행동반자관계
    - TCR || REL_CD <========== tn_companion_info_동반자정보_C

    -> 기본값 0

> 숙소유형 
    - HTY || LODGING_TYPE_CD <========= tn_visit_area_info_방문지정보_C

		-> 기본값 0


> 성별
    - GEN || GENDER  <========== tn_traveller_master_여행객 Master_C

    특이사항 남, 여로 적혀 있음

    해당 갑은 남 : 1, 여 : 0으로 변경

    null 값은 2 : 성별 불명 로 대체
    
    -> 기본값 2

> 여행동기
    - TMT || TRAVEL_MOTiVE_1 ~ 3 <========== tn_traveller_master_여행객 Master_C

    해당 항목이 1~3번 존재

    3가지 중복 없이 입력

		-> 기본값 10

> 여행스타일
    - TSY || TRAVEL_STYL_1 ~ 8  <========== tn_traveller_master_여행객 Master_C

    해당 항목이 1~8번이 존재

    => 1개만 받을 예정    

		-> 기본값 4

> 동반자연령대
    - AGE || COMPANION_AGE_GRP <============ tn_companion_info_동반자정보_C

    -> 기본값 0
    

#################################### 여기 까지는 후에 타 서버에서 받아서 예측시 입력해 줄 데이터 #####################################



#################################################################################################################################

# OUTPUT
 

> (DGSTFN + REVISIT_INTENTOIN + RCMDTN_INTENTION) / 3 =======>>>>>>>>> 실제 출력
    
    모두 tn_visit_area_info_방문지정보_C 에 존재

    '()' 안의 더해지는 값들은 각 값의 범위가 1~

> 여행지 만족도

    train 간에는 모델은 입력된 여행 데이터를 바탕으로 각 여행지의 점수를 맞추는 작업을 수행한다.
        
        출력한 점수 중 상위 k개의 장소를 선정 해당 장소 중에 정답이 존재한다면
    
    predict의 경우 모든 여행지에 대해서 점수를 매기는 작업 수행 -> softmax를 통해서 표준화하고 내림차순 정렬



> 여행 ID  ======>>>>>> 필요 X
    - || TRAVEL_ID <======= tn_traveller_master_여행객 Master_C

> 방문지 주소 # 전처리 후 리스트해서 전달
    - - || LOTNO_ADDR <====== tn_visit_area_info_방문지정보_C

    몇 항목만 사용
        VIS
            1:자연관광지
            2:역사/유적/종교 시설
            3:문화 시설
            4:상업지구
            5:레저
            6:테마시설
            7:산책로, 둘레길 등
            8:지역 축제/행사
            13: 체험 활동 관광지

            보류
                11:식당/카페
                

-------------------------------------------------------

            제거
                10:상점
                12:기타
                24:숙소
                21:집
                22:친구/친구집
                23:사무실
                

2가지 경우가 존재

1. travel id를 확률을 맞추게하고

    2. 확률 분포를 이용 해당 분포 순으로 해당 id가 방문한 지역 전달

2. 방문지 수에 맞게 데이터를 확장한다.

    2. 이 경우 추가적인 데이터를 심어주는게 좋을 듯하다 예를 들어 순서 방문 순서 정보나 재방문 점수 같은 정보를 추가해서 유저 정보 + 다른 추가 정보 이용 학습

In [62]:
input_data_csv_name_list = ["tn_traveller_master_여행객 Master_C", "tn_travel_여행_C", "tn_companion_info_동반자정보_C", "tn_visit_area_info_방문지정보_C"]

In [63]:
output_data_csv_name_list = ["tn_travel_여행_C","tn_visit_area_info_방문지정보_C"]

In [64]:
data_path = "./data/정보대 SW contest/279.국내 여행로그 데이터(서부권)/01.데이터/Training/02.라벨링데이터/TL_csv"
extender = '.csv'

index_dict = {'TRAVEL_ID':['1']}
# input_dict = {'여행객':[],'여행':[],'동반자정보':[],'방문지정보':[]}
# 방문지 정보는 output에서 같이 처리
input_dict = {'여행객':[],'여행':[],'동반자정보':[]}
output_dict = {'TRAVELER_ID':[],'방문지정보':[]}
print(index_dict)
print(input_dict)
print(output_dict)
print(data_path)

{'TRAVEL_ID': ['1']}
{'여행객': [], '여행': [], '동반자정보': []}
{'TRAVELER_ID': [], '방문지정보': []}
./data/정보대 SW contest/279.국내 여행로그 데이터(서부권)/01.데이터/Training/02.라벨링데이터/TL_csv


In [65]:
tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[0]+extender))

index_list = tmp_df['TRAVELER_ID'].values
index_dict['TRAVEL_ID'] = index_list
print(index_dict)
# print(tmp_df['TRAVELER_ID'].values)

{'TRAVEL_ID': array(['c003867', 'c000679', 'c013674', ..., 'c008277', 'c012053',
       'c007519'], dtype=object)}


In [66]:
get_list = ['TRAVELER_ID','INCOME', 'GENDER','HOUSE_INCOME', 'TRAVEL_MOTIVE_1','TRAVEL_MOTIVE_2','TRAVEL_MOTIVE_3', 
            'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8']

tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[0]+extender))
tmp_value = tmp_df['TRAVELER_ID'].values

tmp_value = [ s[0]+"_"+s for s in tmp_value ]
tmp_df['TRAVEL_ID'] = tmp_value
tmp_df = tmp_df.drop('TRAVELER_ID', axis='columns')

tmp_df = tmp_df.set_index("TRAVEL_ID")

input_dict['여행객'] = tmp_df[get_list[1:]]
print(input_dict['여행객'].columns)
print(input_dict['여행객'].head())
print_isna(input_dict['여행객'])


Index(['INCOME', 'GENDER', 'HOUSE_INCOME', 'TRAVEL_MOTIVE_1',
       'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2',
       'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6',
       'TRAVEL_STYL_7', 'TRAVEL_STYL_8'],
      dtype='object')
           INCOME GENDER  HOUSE_INCOME  TRAVEL_MOTIVE_1  TRAVEL_MOTIVE_2  \
TRAVEL_ID                                                                  
c_c003867       4      여           NaN              1.0              7.0   
c_c000679       4      여           NaN              5.0              3.0   
c_c013674       4      여           6.0              1.0              8.0   
c_c010826       1      여           1.0              7.0              2.0   
c_c012020       3      여           7.0              3.0              1.0   

           TRAVEL_MOTIVE_3  TRAVEL_STYL_1  TRAVEL_STYL_2  TRAVEL_STYL_3  \
TRAVEL_ID                                                                 
c_c003867              2.0           

In [67]:
get_list = ['TRAVEL_ID', 'TRAVEL_MISSION']

tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[1]+extender))
tmp_df = tmp_df.set_index(get_list[0])

input_dict['여행'] = tmp_df[get_list[1:]]
print_isna(input_dict['여행'])

TRAVEL_MISSION    0
dtype: int64


In [68]:
get_list = ['TRAVEL_ID', 'REL_CD', 'COMPANION_AGE_GRP']

tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[2]+extender))
tmp_df = tmp_df.set_index(get_list[0])
input_dict['동반자정보'] = tmp_df[get_list[1:]]
print_isna(input_dict['동반자정보'])

REL_CD               0
COMPANION_AGE_GRP    0
dtype: int64


In [69]:
# get_list = ['TRAVEL_ID', 'LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD']

# tmp_df = pd.read_csv(path.join(data_path, input_data_csv_name_list[3]+extender))
# tmp_df = tmp_df.set_index(get_list[0])
# input_dict['방문지정보'] = tmp_df[get_list[1:]]
# # print(tmp_df)

"""output_dict의 방문지정보에서 같이 처리"""

'output_dict의 방문지정보에서 같이 처리'

In [70]:
get_list = ['TRAVEL_ID','TRAVELER_ID']

tmp_df = pd.read_csv(path.join(data_path, output_data_csv_name_list[0]+extender))
tmp_df = tmp_df.set_index(get_list[0])
output_dict['TRAVELER_ID'] = tmp_df[get_list[1:]]
# print(output_dict['TRAVELER_ID'])
print_isna(output_dict['TRAVELER_ID'])

TRAVELER_ID    0
dtype: int64


In [71]:
get_list = ['TRAVEL_ID', 'VISIT_AREA_ID', 'LOTNO_ADDR','LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD','REVISIT_YN','DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'X_COORD','Y_COORD']

tmp_df = pd.read_csv(path.join(data_path, output_data_csv_name_list[1]+extender))
tmp_df = tmp_df.set_index(get_list[0])
print(tmp_df['Y_COORD'])
output_dict['방문지정보'] = tmp_df[get_list[1:]]
print_isna(output_dict['방문지정보'])
# print(tmp_df)

TRAVEL_ID
c_c005733       36.17142
c_c000326      34.742572
c_c003528      36.858789
c_c001050      35.814382
c_c001776      36.544343
                ...     
c_c011285    36.32766877
c_c003324            NaN
c_c001034    36.46550084
c_c004840    35.37052749
c_c002113    35.81700842
Name: Y_COORD, Length: 34785, dtype: object
VISIT_AREA_ID             0
LOTNO_ADDR             8381
LODGING_TYPE_CD       31832
VISIT_AREA_TYPE_CD        0
REVISIT_YN             6823
DGSTFN                 6839
REVISIT_INTENTION      6839
RCMDTN_INTENTION       6839
X_COORD                7330
Y_COORD                7330
dtype: int64


C:\Users\jinmi\AppData\Local\Temp\ipykernel_28752\4009363776.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp_df = pd.read_csv(path.join(data_path, output_data_csv_name_list[1]+extender))


In [72]:
for keys in input_dict:
    print(keys)
    # print(input_dict[keys].head())
    tmp_df = input_dict[keys]
    print("----------------------------------------")
    print(tmp_df.index)
    # print(tmp_df.sort_index().head())
    print("----------------------------------------"*2)


여행객
----------------------------------------
Index(['c_c003867', 'c_c000679', 'c_c013674', 'c_c010826', 'c_c012020',
       'c_c009357', 'c_c012760', 'c_c003756', 'c_c003541', 'c_c002909',
       ...
       'c_c013812', 'c_c012432', 'c_c009023', 'c_c010863', 'c_c001641',
       'c_c005317', 'c_c000381', 'c_c008277', 'c_c012053', 'c_c007519'],
      dtype='object', name='TRAVEL_ID', length=3200)
--------------------------------------------------------------------------------
여행
----------------------------------------
Index(['c_c011995', 'c_c008882', 'c_c003940', 'c_c012638', 'c_c011983',
       'c_c012367', 'c_c013909', 'b_b006673', 'c_c012881', 'c_c002721',
       ...
       'c_c002389', 'c_c003692', 'c_c013085', 'c_c011684', 'c_c000183',
       'c_c012049', 'c_c010871', 'c_c004312', 'c_c009302', 'c_c000104'],
      dtype='object', name='TRAVEL_ID', length=3200)
--------------------------------------------------------------------------------
동반자정보
-------------------------------------

In [73]:
for keys in input_dict:
    print(keys)
    # print(input_dict[keys].head())
    tmp_df = input_dict[keys]
    print("----------------------------------------")
    print(tmp_df.columns)
    # print(tmp_df.sort_index().head())
    print("----------------------------------------"*2)
print("----------------------------------------"*2)
print("----------------------------------------"*2)
print("----------------------------------------"*2)
for keys in output_dict:
    print(keys)
    # print(input_dict[keys].head())
    tmp_df = output_dict[keys]
    print("----------------------------------------")
    print(tmp_df.columns)
    # print(tmp_df.sort_index().head())
    print("----------------------------------------"*2)


여행객
----------------------------------------
Index(['INCOME', 'GENDER', 'HOUSE_INCOME', 'TRAVEL_MOTIVE_1',
       'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2',
       'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6',
       'TRAVEL_STYL_7', 'TRAVEL_STYL_8'],
      dtype='object')
--------------------------------------------------------------------------------
여행
----------------------------------------
Index(['TRAVEL_MISSION'], dtype='object')
--------------------------------------------------------------------------------
동반자정보
----------------------------------------
Index(['REL_CD', 'COMPANION_AGE_GRP'], dtype='object')
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
T

# 방분지 정보 정제

> 아래는 해당 번호들 제외하고 걸러야한다

VIS - VISIT_AREA_TYPE_CD 사용할 항목들

            1:자연관광지
            2:역사/유적/종교 시설
            3:문화 시설
            4:상업지구
            5:레저
            6:테마시설
            7:산책로, 둘레길 등
            8:지역 축제/행사
            11: 식당 카페
            13: 체험 활동 관광지

            


> 아래는 계산해서 SCORE로 추가

(DGSTFN + REVISIT_INTENTOIN + RCMDTN_INTENTION) / 3 =======>>>>>>>>> 실제 출력
    
    모두 tn_visit_area_info_방문지정보_C 에 존재

    '()' 안의 더해지는 값들은 각 값의 범위가 1~5


> 최종 11352개의 방문지 존재 해당 정보를 바탕으로 데이터 수립 

    TRAVERL_ID에 해당 하는 정보들을 추가 적으로 붙여넣어 준다.



In [74]:
tmp_df = output_dict['방문지정보']

tmp_df['Y_COORD'] = tmp_df['Y_COORD'].astype("str")
tmp_df['Y_COORD'] = tmp_df['Y_COORD'].apply(lambda x: ''.join(re.findall(r'\d+', x)))
tmp_df.loc[tmp_df['Y_COORD']=="",'Y_COORD'] = None
tmp_df['Y_COORD'].astype("float")

output_dict['방문지정보'] = tmp_df

In [75]:
tmp_df = output_dict['방문지정보']
print(tmp_df.head())

new_df = tmp_df[tmp_df['VISIT_AREA_TYPE_CD'].isin([1,2,3,4,5,6,7,8,11,13])]
new_df.loc[:,'SCORE'] = ((new_df['DGSTFN'] + new_df['REVISIT_INTENTION'] + new_df['RCMDTN_INTENTION']) / 3.0).round(4)
# print(new_df)



output_dict['방문지정보'] = new_df
# print(output_dict['방문지정보'])

duplicated_index = new_df.index.duplicated()
print(duplicated_index)

           VISIT_AREA_ID             LOTNO_ADDR  LODGING_TYPE_CD  \
TRAVEL_ID                                                          
c_c005733     2210090004  충북 영동군 영동읍 계산리 653-12              NaN   
c_c000326     2208210005         전남 여수시 수정동 772              NaN   
c_c003528     2210080004     충북 진천군 진천읍 읍내리 165              NaN   
c_c001050     2208270005    전북 전주시 완산구 교동 산 1-3              NaN   
c_c001776     2208270009  충남 홍성군 서부면 남당리 767-18              NaN   

           VISIT_AREA_TYPE_CD REVISIT_YN  DGSTFN  REVISIT_INTENTION  \
TRAVEL_ID                                                             
c_c005733                  11          N     4.0                3.0   
c_c000326                  11          N     5.0                5.0   
c_c003528                   8          Y     3.0                4.0   
c_c001050                   2          N     5.0                5.0   
c_c001776                   1          N     4.0                4.0   

           RCMDTN_INTENTI

C:\Users\jinmi\AppData\Local\Temp\ipykernel_28752\2096939357.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[:,'SCORE'] = ((new_df['DGSTFN'] + new_df['REVISIT_INTENTION'] + new_df['RCMDTN_INTENTION']) / 3.0).round(4)


In [76]:
# tmp_df = input_dict['방문지정보']
# print(tmp_df.head())

# new_df = tmp_df[tmp_df['VISIT_AREA_TYPE_CD'].isin([1,2,3,4,5,6,7,8,13])]
# # print(new_df)
# new_df = new_df.drop("VISIT_AREA_TYPE_CD", axis='columns')
# input_dict['방문지정보'] = new_df
# # print(output_dict['방문지정보'])

# duplicated_index = new_df.index.duplicated()
# print(duplicated_index)

# 데이터 이어 붙이기

나머지 정보들을 output_dict['방문자정보']에 적절한 client_code에 이어 붙여 준다.


여행객
----------------------------------------
Index(['INCOME', 'HOUSE_INCOME', 'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2',
       'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
       'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
       'TRAVEL_STYL_8'],
      dtype='object')
--------------------------------------------------------------------------------
여행
----------------------------------------
Index(['TRAVEL_MISSION'], dtype='object')
--------------------------------------------------------------------------------
동반자정보
----------------------------------------
Index(['REL_CD', 'COMPANION_AGE_GRP'], dtype='object')
--------------------------------------------------------------------------------
방문지정보
----------------------------------------
Index(['LODGING_TYPE_CD'], dtype='object')
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
TRAVELER_ID
----------------------------------------
Index(['TRAVELER_ID'], dtype='object')
--------------------------------------------------------------------------------
방문지정보
----------------------------------------
Index(['LOTNO_ADDR', 'VISIT_AREA_TYPE_CD', 'REVISIT_YN', 'DGSTFN',
       'REVISIT_INTENTION', 'RCMDTN_INTENTION'],
      dtype='object')
--------------------------------------------------------------------------------

In [77]:
output_dict['최종'] = output_dict['방문지정보'].copy()

In [78]:
output_dict['최종']
tmp_df = output_dict['최종']
for col_name in output_dict['최종'].columns:
    print(col_name)
    print(pd.isna(tmp_df[col_name]).sum())
    print("=================")

VISIT_AREA_ID
0
LOTNO_ADDR
1604
LODGING_TYPE_CD
19884
VISIT_AREA_TYPE_CD
0
REVISIT_YN
22
DGSTFN
23
REVISIT_INTENTION
23
RCMDTN_INTENTION
23
X_COORD
1022
Y_COORD
1022
SCORE
23


In [79]:
tmp_df = output_dict['최종']
# tmp_df = tmp_df.merge(input_dict['동반자정보'], left_index=True, right_index=True)
tmp_df = tmp_df.join(input_dict['동반자정보'], how='inner', on=['TRAVEL_ID'])
# tmp_df.loc['a_a000202', 'COMPANION_AGE_GRP'] = -1
# print(tmp_df.loc['a_a000202'])
print(output_dict['최종'].shape)
print(tmp_df.shape)
print(input_dict['동반자정보'].shape)
print(pd.isna(input_dict['동반자정보']).sum())
print(tmp_df)
print("***"*10)
print(output_dict)

(19964, 11)
(22721, 13)
(4741, 2)
REL_CD               0
COMPANION_AGE_GRP    0
dtype: int64
           VISIT_AREA_ID                LOTNO_ADDR  LODGING_TYPE_CD  \
TRAVEL_ID                                                             
c_c000326     2208210005            전남 여수시 수정동 772              NaN   
c_c000326     2208200005                       NaN              NaN   
c_c000326     2208200009            전남 여수시 중앙동 468              NaN   
c_c000326     2208220005          전남 여수시 수정동 774-4              NaN   
c_c000326     2208210002          전남 여수시 웅천동 산 274              NaN   
...                  ...                       ...              ...   
c_c014384     2211120002            전남 화순군 화순읍 삼천리              NaN   
c_c012334     2211060004  충북 청주시 상당구 남일면 두산리 290-2              NaN   
c_c012334     2211060004  충북 청주시 상당구 남일면 두산리 290-2              NaN   
c_c012334     2211060004  충북 청주시 상당구 남일면 두산리 290-2              NaN   
c_c013166     2211030004         전남 담양군 무정면 동산리 60     

In [80]:
for keys in input_dict:
    tmp_df = input_dict[keys]
    tmp_col = tmp_df.columns
    # for col_name in tmp_col:
    #     tmp_df[col_name]
    
    if keys=="동반자정보":
        break
    
    print(keys)
    print(output_dict['최종'].shape)
    print(output_dict['최종'].join(tmp_df).shape)
    output_dict['최종'] = output_dict['최종'].join(tmp_df)
    print(output_dict['최종'].columns)
    
        

여행객
(19964, 11)
(19964, 25)
Index(['VISIT_AREA_ID', 'LOTNO_ADDR', 'LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD',
       'REVISIT_YN', 'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION',
       'X_COORD', 'Y_COORD', 'SCORE', 'INCOME', 'GENDER', 'HOUSE_INCOME',
       'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3',
       'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
       'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8'],
      dtype='object')
여행
(19964, 25)
(19964, 26)
Index(['VISIT_AREA_ID', 'LOTNO_ADDR', 'LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD',
       'REVISIT_YN', 'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION',
       'X_COORD', 'Y_COORD', 'SCORE', 'INCOME', 'GENDER', 'HOUSE_INCOME',
       'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3',
       'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
       'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
       'TRAVEL_MISSION'],
      dtype='objec

In [81]:
tmp_df = output_dict["최종"]
output_index_list = list(set(tmp_df.index.to_list()))
input_index_list = list(set(input_dict["동반자정보"].index.to_list()))
# print(output_index_list)
# print(input_index_list)
# print(input_dict["동반자정보"])
# print(input_dict["동반자정보"].loc["c_c000341"])

for index in output_index_list:
    REL_CD = 0
    COMPANION_AGE_GRP = 0
    if index in input_index_list:
        tmp_len = input_dict["동반자정보"].loc[index].shape
        tmp_data = input_dict["동반자정보"].loc[index]
        if len(tmp_len) >= 2:
            tmp_data = tmp_data.iloc[0].to_frame()
        else:
            tmp_data = tmp_data.to_frame()
        # print(index)
        # print(len(tmp_data))
        # print(tmp_data.to_frame())
        # print(tmp_data)
        # print(tmp_data.values[0,0])
        # print(tmp_data.values[1,0])
        tmp_df.loc[index,"REL_CD"] = tmp_data.values[0,0]
        tmp_df.loc[index,"COMPANION_AGE_GRP"]= tmp_data.values[1,0]
        
output_dict['최종'] = tmp_df

In [82]:
print(output_dict['최종'].columns)

Index(['VISIT_AREA_ID', 'LOTNO_ADDR', 'LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD',
       'REVISIT_YN', 'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION',
       'X_COORD', 'Y_COORD', 'SCORE', 'INCOME', 'GENDER', 'HOUSE_INCOME',
       'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3',
       'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
       'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
       'TRAVEL_MISSION', 'REL_CD', 'COMPANION_AGE_GRP'],
      dtype='object')


In [83]:
tmp_df = output_dict['최종']
for col_name in output_dict['최종'].columns:
    print(col_name)
    print(pd.isna(tmp_df[col_name]).sum())
    print("=================")
    

tmp_df.to_csv("./data/tmp_dataset.csv")

VISIT_AREA_ID
0
LOTNO_ADDR
1604
LODGING_TYPE_CD
19884
VISIT_AREA_TYPE_CD
0
REVISIT_YN
22
DGSTFN
23
REVISIT_INTENTION
23
RCMDTN_INTENTION
23
X_COORD
1022
Y_COORD
1022
SCORE
23
INCOME
4003
GENDER
4003
HOUSE_INCOME
8138
TRAVEL_MOTIVE_1
4008
TRAVEL_MOTIVE_2
4625
TRAVEL_MOTIVE_3
5408
TRAVEL_STYL_1
4003
TRAVEL_STYL_2
4003
TRAVEL_STYL_3
4003
TRAVEL_STYL_4
4003
TRAVEL_STYL_5
4003
TRAVEL_STYL_6
4003
TRAVEL_STYL_7
4003
TRAVEL_STYL_8
4003
TRAVEL_MISSION
4132
REL_CD
7368
COMPANION_AGE_GRP
7368


# 데이터 보정

> LOTNO_ADDR

    885

    필수로 존재해야 하는 값이다.
    없으면 해당 행은 모두 제거

=================
> LODGING_TYPE_CD

    11295

    숙소를 쓰는 경우가 흔하지 않은 것인지  na가 많다
    나머지는 모두 0으로 채운다

=================
> VISIT_AREA_TYPE_CD

    0


=================
> REVISIT_YN

    17
    재방문 할 것인지 여부
    yes - 1
    no - 0으로 변경 후 
    0 값으로 나머지 보정

=================
> DGSTFN

    18
    만족도 점수

    필수 값이다
    평균 값으로 대체

=================
> REVISIT_INTENTION
    
    18
    재방문의향

    필수 값이다
    평균 값으로 대체


=================
> RCMDTN_INTENTION

    18
    추천의향이다

    필수 값이다
    평균 값으로 대체

=================
> SCORE

    18
    위의 값으로 계산된 점수 이다

    필수 값이다
    평균 값으로 대체

=================
> INCOME
    
    2345
    개인 소득 수준이다

    0으로 대체

=================
> GENDER
    
    4003

    2로 대체

=================

> HOUSE_INCOME
    
    4581

    사용x 날림

=================
> TRAVEL_MOTIVE_1

    2348

    

# default로 10:기타로 설정

=================
> TRAVEL_MOTIVE_2
    
    2709
=================
> TRAVEL_MOTIVE_3
    
    3128
=================
> TRAVEL_STYL_1
   
    2345

# 앞의 3개의 값만을 사용
# default로 4:중립 사용

=================
> TRAVEL_STYL_2
    
    2345
=================
> TRAVEL_STYL_3
    
    2345
=================
> TRAVEL_STYL_4
    
    2345
=================
> TRAVEL_STYL_5
    
    2345
=================
> TRAVEL_STYL_6
    
    2345
=================
> TRAVEL_STYL_7
    
    2345
=================
> TRAVEL_STYL_8
    
    2345
=================
> TRAVEL_MISSION
    
    2322
    
# 해당 값은 0으로 초기화 적당한 default 값이 존재하지 않음
=================

=================
> REL_CD

    7368

# 0으로 대체
=================
> COMPANION_AGE_GRP

    7368

# 0으로 대체
=================

In [84]:
final_df_bk = tmp_df.copy().reset_index()
final_df = final_df_bk.copy()
print(final_df.index)
print(final_df.columns)

RangeIndex(start=0, stop=19964, step=1)
Index(['TRAVEL_ID', 'VISIT_AREA_ID', 'LOTNO_ADDR', 'LODGING_TYPE_CD',
       'VISIT_AREA_TYPE_CD', 'REVISIT_YN', 'DGSTFN', 'REVISIT_INTENTION',
       'RCMDTN_INTENTION', 'X_COORD', 'Y_COORD', 'SCORE', 'INCOME', 'GENDER',
       'HOUSE_INCOME', 'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3',
       'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
       'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
       'TRAVEL_MISSION', 'REL_CD', 'COMPANION_AGE_GRP'],
      dtype='object')


In [85]:
target_col = "LOTNO_ADDR"

print(final_df.shape)
print_isna(final_df[target_col])
# print(list(final_df["LOTNO_ADDR"].unique()))
tmp_df = final_df

tmp_df = tmp_df.loc[~pd.isna(tmp_df.loc[:,target_col])]



print(tmp_df.shape)
print_isna(tmp_df[target_col])
# print(list(tmp_df["LOTNO_ADDR"].unique()))


(19964, 29)
1604
(18360, 29)
0


In [86]:
final_df = tmp_df

In [87]:
target_col = "X_COORD"

print(final_df.shape)
print_isna(final_df[target_col])
tmp_df = final_df

tmp_df = tmp_df.loc[~pd.isna(tmp_df.loc[:,target_col])]



print(tmp_df.shape)
print_isna(tmp_df[target_col])
print(tmp_df[target_col])
print(tmp_df["Y_COORD"].astype("float"))

(18360, 29)
24
(18336, 29)
0
0        127.253206
1        126.822377
2        127.222808
3        127.224081
4        126.622213
            ...    
19954    127.760061
19955    127.508157
19958    127.123781
19961    127.149958
19963    127.122291
Name: X_COORD, Length: 18336, dtype: float64
0        3.677142e+09
1        3.689023e+09
2        3.678367e+09
3        3.675740e+09
4        3.632814e+09
             ...     
19954    3.495980e+09
19955    3.488734e+08
19958    3.585035e+09
19961    3.581426e+09
19963    3.579709e+09
Name: Y_COORD, Length: 18336, dtype: float64


In [88]:
final_df = tmp_df

In [89]:
target_col = "LODGING_TYPE_CD"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df
tmp_df[target_col] = tmp_df[target_col].fillna(0)

print(tmp_df.shape)
print_isna(tmp_df[target_col])


(18336, 29)
18273
(18336, 29)
0


In [90]:
final_df = tmp_df

In [91]:
target_col = "REVISIT_YN"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df.loc[tmp_df[target_col]=="Y", target_col] = 1
tmp_df.loc[tmp_df[target_col]=="N", target_col] = 0

# tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].max())
tmp_df[target_col] = tmp_df[target_col].fillna(0)

print(tmp_df[target_col])

o = ((tmp_df[target_col]==1).sum())
z = ((tmp_df[target_col]==0).sum())

print(o+z)

print(tmp_df.shape)
print_isna(tmp_df[target_col])

(18336, 29)
16
0        0
1        1
2        0
3        1
4        0
        ..
19954    1
19955    0
19958    0
19961    0
19963    0
Name: REVISIT_YN, Length: 18336, dtype: int64
18336
(18336, 29)
0


In [92]:
final_df = tmp_df

In [93]:
target_col = "DGSTFN"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].mean())

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

(18336, 29)
17
0    4.0
1    4.0
2    4.0
3    2.0
4    4.0
Name: DGSTFN, dtype: float64
(18336, 29)
0


In [94]:
final_df = tmp_df

In [95]:
target_col = "REVISIT_INTENTION"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].mean())

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

(18336, 29)
17
0    3.0
1    4.0
2    4.0
3    3.0
4    4.0
Name: REVISIT_INTENTION, dtype: float64
(18336, 29)
0


In [96]:
final_df = tmp_df

In [97]:
target_col = "RCMDTN_INTENTION"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].mean())

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

(18336, 29)
17
0    3.0
1    4.0
2    4.0
3    3.0
4    4.0
Name: RCMDTN_INTENTION, dtype: float64
(18336, 29)
0


In [98]:
final_df = tmp_df

In [99]:
target_col = "SCORE"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].mean())

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

(18336, 29)
17
0    3.3333
1    4.0000
2    4.0000
3    2.6667
4    4.0000
Name: SCORE, dtype: float64
(18336, 29)
0


In [100]:
final_df = tmp_df

In [101]:
target_col = "INCOME"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

# tmp_df[target_col] = tmp_df[target_col].fillna(tmp_df[target_col].mean())
tmp_df[target_col] = tmp_df[target_col].fillna(0)

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

(18336, 29)
3616
0     0.0
1     0.0
2     0.0
3     0.0
4    12.0
Name: INCOME, dtype: float64
(18336, 29)
0


In [102]:
final_df = tmp_df

In [103]:
target_col = "HOUSE_INCOME"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df
tmp_df = tmp_df.drop(axis=1, columns=target_col)

print(tmp_df.shape)

(18336, 29)
7436
(18336, 28)


In [104]:
final_df = tmp_df

In [105]:
print(final_df.shape)
# print_isna(final_df[target_col])

tmp_df = final_df

for i in range(1,4):
    target_col = "TRAVEL_MOTIVE_" + str(i)

    tmp_df[target_col] = tmp_df[target_col].fillna(10)

    print(tmp_df[target_col].head())

    print((tmp_df[target_col]==10).sum())
    print(tmp_df.shape)
    print_isna(tmp_df[target_col])

(18336, 28)
0    10.0
1    10.0
2    10.0
3    10.0
4     3.0
Name: TRAVEL_MOTIVE_1, dtype: float64
3826
(18336, 28)
0
0    10.0
1    10.0
2    10.0
3    10.0
4     1.0
Name: TRAVEL_MOTIVE_2, dtype: float64
4261
(18336, 28)
0
0    10.0
1    10.0
2    10.0
3    10.0
4    10.0
Name: TRAVEL_MOTIVE_3, dtype: float64
5342
(18336, 28)
0


In [106]:
final_df = tmp_df

In [107]:
tmp_col = [ c for c in tmp_df.columns if "TRAVEL_STYL" in c]
print(tmp_df[tmp_col].describe())
print(tmp_col)
print(tmp_df[tmp_col].mode(axis=0))
print(tmp_df[tmp_col].mode(axis=1))
print(tmp_df[tmp_col].mode(axis=1)[0].value_counts())
tmp_df['TRAVEL_STYL_0'] = tmp_df[tmp_col].mode(axis=1)[0]


       TRAVEL_STYL_1  TRAVEL_STYL_2  TRAVEL_STYL_3  TRAVEL_STYL_4  \
count   14720.000000   14720.000000   14720.000000   14720.000000   
mean        3.024253       2.625679       2.534443       3.416916   
std         1.622175       1.635048       1.497525       1.590540   
min         1.000000       1.000000       1.000000       1.000000   
25%         2.000000       1.000000       1.000000       2.000000   
50%         3.000000       2.000000       2.000000       3.000000   
75%         4.000000       4.000000       3.000000       4.000000   
max         7.000000       7.000000       7.000000       7.000000   

       TRAVEL_STYL_5  TRAVEL_STYL_6  TRAVEL_STYL_7  TRAVEL_STYL_8  
count   14720.000000   14720.000000   14720.000000   14720.000000  
mean        3.681726       3.730435       3.804688       4.787568  
std         1.766364       1.656491       1.994003       1.759675  
min         1.000000       1.000000       1.000000       1.000000  
25%         2.000000       2.000000   

In [108]:
print(final_df.shape)
# print_isna(final_df[target_col])

tmp_df = final_df

for i in range(0,9):
    target_col = "TRAVEL_STYL_" + str(i)

    print((tmp_df[target_col]==4).sum())
    
    tmp_df[target_col] = tmp_df[target_col].fillna(4)

    print(tmp_df[target_col].head())

    print((tmp_df[target_col]==4).sum())
    print(tmp_df.shape)
    print_isna(tmp_df[target_col])

(18336, 29)
2304
0    4.0
1    4.0
2    4.0
3    4.0
4    4.0
Name: TRAVEL_STYL_0, dtype: float64
5920
(18336, 29)
0
3180
0    4.0
1    4.0
2    4.0
3    4.0
4    1.0
Name: TRAVEL_STYL_1, dtype: float64
6796
(18336, 29)
0
2205
0    4.0
1    4.0
2    4.0
3    4.0
4    4.0
Name: TRAVEL_STYL_2, dtype: float64
5821
(18336, 29)
0
2058
0    4.0
1    4.0
2    4.0
3    4.0
4    4.0
Name: TRAVEL_STYL_3, dtype: float64
5674
(18336, 29)
0
3405
0    4.0
1    4.0
2    4.0
3    4.0
4    4.0
Name: TRAVEL_STYL_4, dtype: float64
7021
(18336, 29)
0
3146
0    4.0
1    4.0
2    4.0
3    4.0
4    1.0
Name: TRAVEL_STYL_5, dtype: float64
6762
(18336, 29)
0
3478
0    4.0
1    4.0
2    4.0
3    4.0
4    4.0
Name: TRAVEL_STYL_6, dtype: float64
7094
(18336, 29)
0
1609
0    4.0
1    4.0
2    4.0
3    4.0
4    1.0
Name: TRAVEL_STYL_7, dtype: float64
5225
(18336, 29)
0
2411
0    4.0
1    4.0
2    4.0
3    4.0
4    4.0
Name: TRAVEL_STYL_8, dtype: float64
6027
(18336, 29)
0


In [109]:
tmp_col = [ c for c in tmp_df.columns if "TRAVEL_STYL" in c]
tmp_col.append(tmp_df.columns[-1])
print(tmp_df[tmp_col].describe())
print(tmp_col)
print(tmp_df[tmp_col].mode(axis=0))
print(tmp_df[tmp_col].mode(axis=1))
print(tmp_df[tmp_col].mode(axis=1)[0].value_counts())


       TRAVEL_STYL_1  TRAVEL_STYL_2  TRAVEL_STYL_3  TRAVEL_STYL_4  \
count   18336.000000   18336.000000   18336.000000   18336.000000   
mean        3.216678       2.896706       2.823462       3.531904   
std         1.504400       1.563707       1.462998       1.443856   
min         1.000000       1.000000       1.000000       1.000000   
25%         2.000000       2.000000       2.000000       2.000000   
50%         4.000000       3.000000       3.000000       4.000000   
75%         4.000000       4.000000       4.000000       4.000000   
max         7.000000       7.000000       7.000000       7.000000   

       TRAVEL_STYL_5  TRAVEL_STYL_6  TRAVEL_STYL_7  TRAVEL_STYL_8  \
count   18336.000000   18336.000000   18336.000000   18336.000000   
mean        3.744492       3.783595       3.843205       4.632253   
std         1.587687       1.488055       1.788278       1.607476   
min         1.000000       1.000000       1.000000       1.000000   
25%         2.000000       3.0000

In [110]:
tmp_col = tmp_df.columns
print(tmp_col)
new_col = ['TRAVEL_ID','VISIT_AREA_ID', 'GENDER','LOTNO_ADDR', 'LODGING_TYPE_CD', 'VISIT_AREA_TYPE_CD',
       'REVISIT_YN', 'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION',
       'SCORE', 'INCOME', 'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2',
       'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_0', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
       'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
       'TRAVEL_STYL_8', 'TRAVEL_MISSION', 'REL_CD', 'COMPANION_AGE_GRP', 'X_COORD', 'Y_COORD']
tmp_df = tmp_df[new_col]
tmp_col = tmp_df.columns
print(tmp_col)


Index(['TRAVEL_ID', 'VISIT_AREA_ID', 'LOTNO_ADDR', 'LODGING_TYPE_CD',
       'VISIT_AREA_TYPE_CD', 'REVISIT_YN', 'DGSTFN', 'REVISIT_INTENTION',
       'RCMDTN_INTENTION', 'X_COORD', 'Y_COORD', 'SCORE', 'INCOME', 'GENDER',
       'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3',
       'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
       'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
       'TRAVEL_MISSION', 'REL_CD', 'COMPANION_AGE_GRP', 'TRAVEL_STYL_0'],
      dtype='object')
Index(['TRAVEL_ID', 'VISIT_AREA_ID', 'GENDER', 'LOTNO_ADDR', 'LODGING_TYPE_CD',
       'VISIT_AREA_TYPE_CD', 'REVISIT_YN', 'DGSTFN', 'REVISIT_INTENTION',
       'RCMDTN_INTENTION', 'SCORE', 'INCOME', 'TRAVEL_MOTIVE_1',
       'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_0', 'TRAVEL_STYL_1',
       'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5',
       'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MISSION',
       'REL_CD', 'C

In [111]:
final_df = tmp_df

In [112]:
target_col = "TRAVEL_MISSION"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(0)

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

(18336, 29)
3769
0    2;
1    2;
2    2;
3    2;
4     0
Name: TRAVEL_MISSION, dtype: object
(18336, 29)
0


In [113]:
final_df = tmp_df

In [114]:
target_col = "REL_CD"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(0)

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])

(18336, 29)
6773
0    2.0
1    2.0
2    2.0
3    2.0
4    7.0
Name: REL_CD, dtype: float64
(18336, 29)
0


In [115]:
final_df = tmp_df

In [116]:
target_col = "COMPANION_AGE_GRP"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(0)

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])
print(tmp_df["LOTNO_ADDR"])

(18336, 29)
6773
0    1.0
1    1.0
2    1.0
3    1.0
4    5.0
Name: COMPANION_AGE_GRP, dtype: float64
(18336, 29)
0
0        충남 천안시 동남구 북면 연춘리 243-40
1            충남 당진시 신평면 운정리 208-1
2        충남 천안시 동남구 목천읍 남화리 230-1
3          충남 천안시 동남구 성남면 용원리 672
4               충남 보령시 명천동 765-40
                   ...           
19954          전남 광양시 태인동 1632-13
19955            전남 순천시 대대동 217-1
19958     전북 전주시 덕진구 덕진동1가 1324-3
19961        전북 전주시 완산구 풍남동3가 102
19963      전북 전주시 완산구 삼천동1가 620-1
Name: LOTNO_ADDR, Length: 18336, dtype: object


In [117]:
final_df = tmp_df

In [118]:
target_col = "GENDER"

print(final_df.shape)
print_isna(final_df[target_col])

tmp_df = final_df

tmp_df[target_col] = tmp_df[target_col].fillna(2)
tmp_df.loc[tmp_df[target_col]=="남", target_col] = 1
tmp_df.loc[tmp_df[target_col]=="여", target_col] = 0

print(tmp_df[target_col].head())

print(tmp_df.shape)
print_isna(tmp_df[target_col])


(18336, 29)
3616
0    2
1    2
2    2
3    2
4    1
Name: GENDER, dtype: object
(18336, 29)
0


In [119]:
final_df = tmp_df

In [120]:
tmp_df = final_df.copy()
tmp_df["TRAVEL_MISSION_1"]=0
tmp_df["TRAVEL_MISSION_2"]=0
tmp_df["TRAVEL_MISSION_3"]=0

print(tmp_df['TRAVEL_MISSION'])
# print(tmp_df['TRAVEL_MISSION'].str.split(';').str[0])

tmp_df['TRAVEL_MISSION_1'] = tmp_df['TRAVEL_MISSION'].str.split(';').str[0]
tmp_df['TRAVEL_MISSION_2'] = tmp_df['TRAVEL_MISSION'].str.split(';').str[1]
tmp_df['TRAVEL_MISSION_3'] = tmp_df['TRAVEL_MISSION'].str.split(';').str[2]

# tmp_df[tmp_df['TRAVEL_MISSION_1']==""]['TRAVEL_MISSION_1']
# print(tmp_df["TRAVEL_MISSION_2"])
# print(tmp_df[tmp_df['TRAVEL_MISSION_2']==""]['TRAVEL_MISSION_2'])
# tmp_df = tmp_df['TRAVEL_MISSION_2'].replace("","0")
# print(tmp_df[tmp_df['TRAVEL_MISSION_2']==""]['TRAVEL_MISSION_2'])
# tmp_df[tmp_df['TRAVEL_MISSION_3']==""]['TRAVEL_MISSION_3']

tmp_df['TRAVEL_MISSION_1'] = tmp_df['TRAVEL_MISSION_1'].fillna(0)
tmp_df['TRAVEL_MISSION_2'] = tmp_df['TRAVEL_MISSION_2'].fillna(0)
tmp_df['TRAVEL_MISSION_3'] = tmp_df['TRAVEL_MISSION_3'].fillna(0)



0                                                       2;
1                                                       2;
2                                                       2;
3                                                       2;
4                                                        0
                               ...                        
19954    1;3;3;4;5;6;28;24;3;11;12;13;22;21;23;24;26;27...
19955    1;3;3;4;5;6;28;24;3;11;12;13;22;21;23;24;26;27...
19958                                                    0
19961                                                    0
19963                                                    0
Name: TRAVEL_MISSION, Length: 18336, dtype: object


In [121]:
final_df = tmp_df

In [122]:
tmp_df = final_df
print(tmp_df.columns)
for col_name in final_df.columns:
    print(col_name)
    print(pd.isna(tmp_df[col_name]).sum())
    print("=================")
    
new_col = ['TRAVEL_ID','VISIT_AREA_ID', 'GENDER','INCOME', 'TRAVEL_MISSION', 'REL_CD', 'LODGING_TYPE_CD',
        'TRAVEL_MISSION_1', 'TRAVEL_MISSION_2', 'TRAVEL_MISSION_3',         
        'TRAVEL_MOTIVE_1', 'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3',
        'TRAVEL_STYL_0', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
        'TRAVEL_STYL_5', 'TRAVEL_STYL_6','TRAVEL_STYL_7', 'TRAVEL_STYL_8',
        'COMPANION_AGE_GRP',
        'REVISIT_YN', 'DGSTFN','REVISIT_INTENTION', 'RCMDTN_INTENTION','SCORE',
        'LOTNO_ADDR',  'VISIT_AREA_TYPE_CD', 'X_COORD', 'Y_COORD']
tmp_df = tmp_df[new_col]
tmp_df.to_csv("./travel_dataset.csv")


Index(['TRAVEL_ID', 'VISIT_AREA_ID', 'GENDER', 'LOTNO_ADDR', 'LODGING_TYPE_CD',
       'VISIT_AREA_TYPE_CD', 'REVISIT_YN', 'DGSTFN', 'REVISIT_INTENTION',
       'RCMDTN_INTENTION', 'SCORE', 'INCOME', 'TRAVEL_MOTIVE_1',
       'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_0', 'TRAVEL_STYL_1',
       'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5',
       'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'TRAVEL_MISSION',
       'REL_CD', 'COMPANION_AGE_GRP', 'X_COORD', 'Y_COORD', 'TRAVEL_MISSION_1',
       'TRAVEL_MISSION_2', 'TRAVEL_MISSION_3'],
      dtype='object')
TRAVEL_ID
0
VISIT_AREA_ID
0
GENDER
0
LOTNO_ADDR
0
LODGING_TYPE_CD
0
VISIT_AREA_TYPE_CD
0
REVISIT_YN
0
DGSTFN
0
REVISIT_INTENTION
0
RCMDTN_INTENTION
0
SCORE
0
INCOME
0
TRAVEL_MOTIVE_1
0
TRAVEL_MOTIVE_2
0
TRAVEL_MOTIVE_3
0
TRAVEL_STYL_0
0
TRAVEL_STYL_1
0
TRAVEL_STYL_2
0
TRAVEL_STYL_3
0
TRAVEL_STYL_4
0
TRAVEL_STYL_5
0
TRAVEL_STYL_6
0
TRAVEL_STYL_7
0
TRAVEL_STYL_8
0
TRAVEL_MISSION
0
REL_CD
0
COMPANION_

In [123]:
tmp_df = pd.read_csv('./travel_dataset.csv')
tmp_df = tmp_df.fillna(0)
print(tmp_df.columns)
for col_name in final_df.columns:
    print(col_name)
    print(pd.isna(tmp_df[col_name]).sum())
    print("=================")
    
tmp_df.to_csv("./travel_dataset.csv")
    

Index(['Unnamed: 0', 'TRAVEL_ID', 'VISIT_AREA_ID', 'GENDER', 'INCOME',
       'TRAVEL_MISSION', 'REL_CD', 'LODGING_TYPE_CD', 'TRAVEL_MISSION_1',
       'TRAVEL_MISSION_2', 'TRAVEL_MISSION_3', 'TRAVEL_MOTIVE_1',
       'TRAVEL_MOTIVE_2', 'TRAVEL_MOTIVE_3', 'TRAVEL_STYL_0', 'TRAVEL_STYL_1',
       'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5',
       'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8', 'COMPANION_AGE_GRP',
       'REVISIT_YN', 'DGSTFN', 'REVISIT_INTENTION', 'RCMDTN_INTENTION',
       'SCORE', 'LOTNO_ADDR', 'VISIT_AREA_TYPE_CD', 'X_COORD', 'Y_COORD'],
      dtype='object')
TRAVEL_ID
0
VISIT_AREA_ID
0
GENDER
0
LOTNO_ADDR
0
LODGING_TYPE_CD
0
VISIT_AREA_TYPE_CD
0
REVISIT_YN
0
DGSTFN
0
REVISIT_INTENTION
0
RCMDTN_INTENTION
0
SCORE
0
INCOME
0
TRAVEL_MOTIVE_1
0
TRAVEL_MOTIVE_2
0
TRAVEL_MOTIVE_3
0
TRAVEL_STYL_0
0
TRAVEL_STYL_1
0
TRAVEL_STYL_2
0
TRAVEL_STYL_3
0
TRAVEL_STYL_4
0
TRAVEL_STYL_5
0
TRAVEL_STYL_6
0
TRAVEL_STYL_7
0
TRAVEL_STYL_8
0
TRAVEL_MISSION
0
REL_C